# Plotting time series (webgl flikering issue)

In [ ]:
import math
import time
import numpy as np
from IPython.display import clear_output, display

In [ ]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook 
from bokeh.models import ColumnDataSource, DatetimeTickFormatter

output_notebook()

### LivePlot class

In [ ]:
class RingBuffer(np.ndarray):
    """
    a multidimensional ring buffer
    see https://gist.github.com/mtambos/aa435461084b5c0025d1
    """

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def __array_finalize__(self, obj):
        if obj is None: return

    def __array_wrap__(self, out_arr, context=None):
        return np.ndarray.__array_wrap__(self, out_arr, context)

    def append(self, x):
        """adds element x to the ring buffer"""
        self[:-1] = self[1:]
        self[-1] = x

In [ ]:
class LivePlot(object):
    
    def __init__(self, doc, enable_webgl):
        self._doc = doc
        self._enable_webgl = enable_webgl
        self._np = 128
        self._cnt = -1
        self._callback_period = 0.1
        array = np.empty((self._np,))
        array.fill(np.nan)
        self._data_buffer = RingBuffer(array)
        array = np.empty((self._np,), dtype=float)
        array.fill(np.nan)
        self._time_buffer = RingBuffer(array)
        columns = dict()
        columns['x'] = np.zeros(1) 
        columns['y'] = np.zeros(1)
        self._cds = ColumnDataSource(data=columns)
        self.show()

    def __gen_data(self): 
        try:
            self._cnt += 1
            y = self._cnt % 16
            self._time_buffer.append(time.time() * 1000.)
            self._data_buffer.append(y)
            self._cds.data.update(x=self._time_buffer, y=self._data_buffer)
            self._cnt = y
        except Exception as e:
            print(e)
    
    def show(self):
        fkwargs = dict()
        # setting output_backend to "webgl" causes the plot to flicker when the data is updated 
        if self._enable_webgl:
            fkwargs['output_backend'] = 'webgl'
        fkwargs['plot_width'] = 950
        fkwargs['plot_height'] = 200
        f = figure(**fkwargs)
        dtf = DatetimeTickFormatter()
        dtf.milliseconds = "%M:%S:%3N"
        dtf.seconds = "%H:%M:%S"
        dtf.minutes = "%H:%M:%S"
        dtf.hours = "%H:%M:%S"
        dtf.days = "%d:%H:%M"
        f.xaxis.formatter = dtf
        f.xaxis.major_label_orientation = math.pi / 4
        f.toolbar_location = 'above'
        f.line(x='x', y='y', source=self._cds, color="navy", alpha=0.5)
        self.__periodic_callback = self._doc.add_periodic_callback(self.__periodic_job, 1000. * self._callback_period)
        self._doc.add_root(f)

    @property
    def num_points(self):
        return self._np
     
    @num_points.setter
    def num_points(self, new_value):
        self._np = int(new_value)
    
    @property
    def callback_period(self):
        return self._callback_period
     
    @callback_period.setter
    def callback_period(self, new_value):
        self._callback_period = new_value
        self._doc.remove_periodic_callback(self.__periodic_callback)
        self.__periodic_callback = self._doc.add_periodic_callback(self.__periodic_job, 1000. * self._callback_period)
        
    def __periodic_job(self):
        self.__gen_data()
        
    def close(self):
        self._doc.add_next_tick_callback(self.__safe_close)
   
    def __safe_close(self):
        self._doc.remove_periodic_callback(self.__periodic_callback)
        self._doc.clear()
        clear_output()

### LivePlotController class

In [ ]:
import ipywidgets as widgets

class LivePlotController(object):
    """a LivePlot controller using ipywidgets"""

    def __init__(self, lp1, lp2):
        self._live_plots = [lp1, lp2]
        self._controls = None
        self.__setup_controls()
        
    @staticmethod
    def l01a(width='auto', *args, **kwargs):
        return widgets.Layout(flex='0 1 auto', width=width, *args, **kwargs)

    @staticmethod
    def l11a(width='auto', *args, **kwargs):
        return widgets.Layout(flex='1 1 auto', width=width, *args, **kwargs)

    def __setup_update_period_slider(self):
        return widgets.FloatSlider(
            value=self._live_plots[0].callback_period,
            min=0.1,
            max=2,
            step=0.1,
            description='Update Period [s]',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            style={'description_width':'initial'}
        )
    
    def __setup_controls(self, **kwargs):
        up_slider = self.__setup_update_period_slider()
        up_slider.observe(self.__on_update_period_changed, names='value')
        close_button = widgets.Button(description="Close", layout=self.l01a(width="100px"))
        close_button.on_click(self.__on_close_clicked)
        self._controls = widgets.HBox([up_slider, close_button], layout=self.l01a())
        display(self._controls)

    def __on_update_period_changed(self, event):
        try:
            #print("__on_update_period_changed: {}".format(event))
            for lp in self._live_plots:
                lp.callback_period = event['new']
        except Exception as e:
            print(e)
            
    def __on_close_clicked(self, b=None):
        self._controls.close()
        for lp in self._live_plots:
            lp.close()

### Embedded Bokeh Server (this is one of the ways to impl async. activity with bokeh)

In [ ]:
import logging
from bokeh.plotting import show
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler

logging.getLogger('bokeh.server.server').setLevel(logging.ERROR)

def setup_document(doc):
    LivePlotController(LivePlot(doc, enable_webgl=True), LivePlot(doc, enable_webgl=False))
    return doc
      
app = Application(FunctionHandler(setup_document))

show(app)